## Load Libraries & Datasets

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from scipy.io import savemat

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from keras.utils.np_utils import to_categorical

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten
from keras import optimizers
from keras import metrics

from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample

In [3]:
def prepare_dset(dset):
    dset = pd.concat([pd.DataFrame(np.asarray(dset.conc > 0.01, dtype='int')).rename(columns={0:'label'}), 
           dset.iloc[:, 3:]], axis=1)
    return dset

In [25]:
## raw data
pbno3_b1_raw = pd.read_csv("./data_final/pbno3_dset/new/pbno3_batch1.csv")
pbno3_b2_raw = pd.read_csv("./data_final/pbno3_dset/new/pbno3_batch2.csv")
pbno3_b1_raw = prepare_dset(pbno3_b1_raw)
pbno3_b2_raw = prepare_dset(pbno3_b2_raw)

## Power Spectrum Normalized
pbno3_b1_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_psn_batch1.csv")
pbno3_b2_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_psn_batch2.csv")
pbno3_b1_psn = prepare_dset(pbno3_b1_psn)
pbno3_b2_psn = prepare_dset(pbno3_b2_psn)

## Baseline Corrected
pbno3_b1_bc = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_batch1.csv")
pbno3_b2_bc = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_batch2.csv")
pbno3_b1_bc = prepare_dset(pbno3_b1_bc)
pbno3_b2_bc = prepare_dset(pbno3_b2_bc)

## Batch Normalized
pbno3_b1_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bn_batch1.csv")
pbno3_b2_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bn_batch2.csv")
pbno3_b1_bn = prepare_dset(pbno3_b1_bn)
pbno3_b2_bn = prepare_dset(pbno3_b2_bn)

## Baseline Correctd & Power Spectrum Normalized
pbno3_b1_bl_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_psn_batch1.csv")
pbno3_b2_bl_psn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_psn_batch2.csv")
pbno3_b1_bl_psn = prepare_dset(pbno3_b1_bl_psn)
pbno3_b2_bl_psn = prepare_dset(pbno3_b2_bl_psn)

## Baseline Correctd & Batch Normalized
pbno3_b1_bl_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_bn_batch1.csv")
pbno3_b2_bl_bn = pd.read_csv("./data_final/pbno3_dset/new/pbno3_bl_bn_batch2.csv")
pbno3_b1_bl_bn = prepare_dset(pbno3_b1_bl_bn)
pbno3_b2_bl_bn = prepare_dset(pbno3_b2_bl_bn)

## Define Util Functions

In [5]:
## Define performance measures
def yoden_index(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
    return j

def pmeasure(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn )
    specificity = tn / (tn + fp)
    f1score = (2 * tp) / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,mean_Stats, sd_Stats, sigfig):
    print(msg.upper())
    print(70*'-')
    print('Accuracy:{} + {}'          .format(round(mean_Stats[0],sigfig), round(sd_Stats[0],sigfig)))
    print('Sensitivity:{} + {} '      .format(round(mean_Stats[1],sigfig), round(sd_Stats[1],sigfig)))
    print('Specificity:{} + {}'       .format(round(mean_Stats[2],sigfig), round(sd_Stats[2],sigfig)))
    print('F1-Score:{} + {}'          .format(round(mean_Stats[3],sigfig), round(sd_Stats[3],sigfig)))
    print('MCC:{} + {}'               .format(round(mean_Stats[4],sigfig), round(sd_Stats[4],sigfig)))
    print('Balance Accuracy:{} + {}'  .format(round(mean_Stats[5],sigfig), round(sd_Stats[5],sigfig)))
    print('Youden-Index:{} + {}'      .format(round(mean_Stats[6],sigfig), round(sd_Stats[6],sigfig)))
    print('AUC:{} + {}'               .format(round(mean_Stats[7],sigfig), round(sd_Stats[7],sigfig)))
    print('AUPR:{} + {}'              .format(round(mean_Stats[8],sigfig), round(sd_Stats[8],sigfig)))  
    print(70*'-')

def Calculate_Stats(y_actual,y_pred, y_score):
    acc = accuracy_score(y_actual, y_pred)
    sen = pmeasure(y_actual, y_pred)['Sensitivity']
    spe = pmeasure(y_actual, y_pred)['Specificity']
    f1 = pmeasure(y_actual, y_pred)['F1-Score']
    mcc = matthews_corrcoef(y_actual, y_pred)
    bacc = balanced_accuracy_score(y_actual, y_pred)
    yi = yoden_index(y_actual, y_pred)
    #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  
    pre, rec, _ = precision_recall_curve(y_actual, y_score, pos_label=1)
    fpr, tpr, _ = roc_curve(y_actual, y_score, pos_label=1)
    auroc = auc(fpr, tpr)
    aupr = auc(rec, pre)
    
    return acc, sen, spe, f1, mcc, bacc, yi, auroc, aupr  

def label_by_th(y_pred, threshold=0.5):
    y_pred_copy = y_pred.copy()
    y_pred_copy[y_pred>= threshold] = 1 
    y_pred_copy[y_pred<threshold] = 0 
    return y_pred_copy

def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [6]:
def get_perf_summary(dataset, model, type):
    ## Prepare dataset
    dataset.loc[dataset.label>=1,"label"]=1
    
    ## Datasets 
    [X, y_true] = np.array(dataset.iloc[:,1:]), np.array(dataset.iloc[:,0])
    y_true = to_categorical(y_true)
    if type=="regression":
        y_score = model.predict(X, batch_size=1800, verbose=0)    
    else:
        y_score = model.predict(X, batch_size=1800, verbose=0)[:,1]
    #y_score = model.predict(X,batch_size=1800, verbose=0)
    #y_pred = to_categorical(get_label(y_score))      

    # Optimal Threshold
    fpr, tpr, thresholds_AUC = roc_curve(y_true.argmax(axis=1), y_score)
    precision, recall, thresholds_AUPR = precision_recall_curve(y_true.argmax(axis=1),y_score)

    ## Optimal Threshold metrics
    distance = (1-fpr)**2+(1-tpr)**2
    EERs = (1-recall)/(1-precision)
    positive = sum(y_true.argmax(axis=1))
    negative = y_true.shape[0]-positive
    ratio = negative/positive
    opt_t_AUC = thresholds_AUC[np.argmin(distance)]
    opt_t_AUPR = thresholds_AUPR[np.argmin(np.abs(EERs-ratio))]
    opt_yodens_j = cutoff_youdens_j(fpr, tpr, thresholds_AUC)        
    y_pred_th = label_by_th(y_score, opt_yodens_j)
    y_pred = to_categorical(y_pred_th)

    print(confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1), labels=[0,1]).ravel())    

    ## Measures
    t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_true, y_pred, y_score)

    print("ACC: {}, Sen: {}, Spe: {}, F1: {}, MCC: {}, BACC: {}, YI: {}, AUC: {}, AUPR: {}".format(t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr))

In [27]:
def get_scale_dset(dset):
    res =[]
    tmp_dset = dset.iloc[:,1:]
    for i, row in tmp_dset.iterrows():
        res.append(row/max(abs(row)))
    
    res = pd.DataFrame(res).reset_index(drop=True)
    res = pd.concat([dset.iloc[:,0], res], axis=1)
    
    return res 

## Define Classifiers

In [43]:
def get_clf_res_ML(dev_dset, ind_test_dset, method):
    num_Trials = 10    
    dev_dset = get_scale_dset(dev_dset)
    ind_test_dset = get_scale_dset(ind_test_dset)

    # import warnings filter
    from warnings import simplefilter
    # ignore all future warnings
    simplefilter(action='ignore', category=FutureWarning)
    [DataX, LabelY] = np.array(dev_dset.iloc[:,1:]), np.array(dev_dset.iloc[:,0])    

    Stats =[]
    Stats_Ind =[]
    for j in range(0, num_Trials):        
        X_train, X_val, y_train, y_val =\
        train_test_split(DataX, LabelY, test_size=0.2, stratify = LabelY)
        X_val, X_test, y_val, y_test =\
        train_test_split(X_val, y_val, test_size=0.5, stratify = y_val)

        if method == "LR":
            clf = LogisticRegression(random_state=0).fit(X_train, y_train)

        elif method == "LinSVM":
            clf = SVC(kernel = 'linear', probability=True).fit(X_train, y_train)

        elif method == "NB":
            clf = GaussianNB().fit(X_train, y_train)

        elif method == "DT":
            clf = DecisionTreeClassifier().fit(X_train, y_train)

        elif method == "MLP":
            clf = MLPClassifier().fit(X_train, y_train)

        elif method == "RF":
            clf = RandomForestClassifier().fit(X_train, y_train)

        elif method == "RBFSVM":
            clf = SVC(kernel = 'rbf', probability=True).fit(X_train, y_train)

        else:
            print("Please check avilable models")
            break;

        y_train_pred = clf.predict(X_train)
        y_train_score = clf.predict_proba(X_train)     
        y_train_pred = to_categorical(y_train_score.argmax(axis=1))
        y_train_score = y_train_score[:,1]

        y_val_pred = clf.predict(X_val)
        y_val_score = clf.predict_proba(X_val)     
        y_val_pred = to_categorical(y_val_score.argmax(axis=1))
        y_val_score = y_val_score[:,1]

        y_test_pred = clf.predict(X_test)
        y_test_score = clf.predict_proba(X_test) 
        y_test_pred = to_categorical(y_test_score.argmax(axis=1))
        y_test_score = y_test_score[:,1] 

        print(confusion_matrix(y_test, y_test_pred.argmax(axis=1), labels=[0,1]).ravel())

        y_train_pred = y_train_pred.argmax(axis=1)
        y_val_pred = y_val_pred.argmax(axis=1)
        y_test_pred = y_test_pred.argmax(axis=1)

        ## Training Measures
        tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr = Calculate_Stats(y_train,y_train_pred, y_train_score);

        ## Validation Measures
        v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr = Calculate_Stats(y_val,y_val_pred, y_val_score);

        ## Test Measures
        t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr = Calculate_Stats(y_test,y_test_pred, y_test_score);

        Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr,
                      v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, v_aupr,
                      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, t_aupr])  

        print(' \nTraining/ Validation / Test BACC :', tr_bacc,'/',v_bacc,'/',t_bacc,
              ' \nTraining/ Validation / Test Youden-index:', tr_yi,'/',v_yi,'/',t_yi,
              ' \nTraining/ Validation / Test MCC:', tr_mcc,'/',v_mcc,'/',t_mcc,
              ' \nTraining/ Validation / Test AUC:', tr_auc,'/',v_auc,'/',t_auc,
              ' \nTraining/ Validation / Test AUPR:', tr_aupr,'/',v_aupr,'/',t_aupr)    

        ## Independent Test
        [Xt1, Yt1] = np.array(ind_test_dset.iloc[:,1:]), np.array(ind_test_dset.iloc[:,0])

        Yt1_pred = clf.predict(Xt1)
        Yt1_score = clf.predict_proba(Xt1)

        Stats_Ind.append(Calculate_Stats(Yt1, Yt1_pred, Yt1_score[:,1]))

        print(Stats_Ind)
    
    return Stats, Stats_Ind

In [9]:
def format_perf_mat(perf_list, model_type):
    perf_tbl = pd.DataFrame(perf_list)
    perf_tbl = pd.concat([perf_tbl, pd.DataFrame(np.repeat(model_type, len(perf_list)))], axis=1)
    perf_tbl.columns = ['Accuracy', "Sensitivity", "Specificity", "F1", "MCC", "BACC", 
                        "Yoden Index", "AUROC", "AUPR", "Model"]    
    return perf_tbl

In [10]:
def get_all_clf_res(devset, indset, species, prep_method):
    NB_cv, NB_ind = get_clf_res_ML(devset, indset, method='NB')
    DT_cv, DT_ind = get_clf_res_ML(devset, indset, method='DT')
    LR_cv, LR_ind = get_clf_res_ML(devset, indset, method='LR')
    RF_cv, RF_ind = get_clf_res_ML(devset, indset, method='RF')
    LinSVM_cv, LinSVM_ind = get_clf_res_ML(devset, indset, method='LinSVM')
    RBFSVM_cv, RBFSVM_ind = get_clf_res_ML(devset, indset, method='RBFSVM')
    MLP_cv, MLP_ind = get_clf_res_ML(devset, indset, method='MLP')
    
    fold_ind_perf = pd.concat([format_perf_mat(NB_ind, 'NB'), 
                               format_perf_mat(DT_ind, 'DT'), 
                               format_perf_mat(LR_ind, 'LR'),
                               format_perf_mat(RF_ind, 'RF'), 
                               format_perf_mat(LinSVM_ind, 'LinSVM'),
                               format_perf_mat(RBFSVM_ind, 'RBFSVM'),
                               format_perf_mat(MLP_ind, 'MLP')], axis=0)

    return fold_ind_perf

## Train and Evaluate Performance of Models

In [29]:
perf_raw_pbno3_1 = get_all_clf_res(pbno3_b1_raw, pbno3_b2_raw, 'pbno3_b1b2', 'raw')
perf_raw_pbno3_2 = get_all_clf_res(pbno3_b2_raw, pbno3_b1_raw, 'pbno3_b2b1', 'raw')

[ 47   3   0 150]
 
Training/ Validation / Test BACC : 0.985 / 0.99 / 0.97  
Training/ Validation / Test Youden-index: 0.97 / 0.98 / 0.94  
Training/ Validation / Test MCC: 0.9799979793876926 / 0.986666070043373 / 0.9599836599916586  
Training/ Validation / Test AUC: 0.9862500000000001 / 0.99 / 0.98  
Training/ Validation / Test AUPR: 0.9954582989265071 / 0.9966887417218543 / 0.993421052631579
[(0.6855, 0.7153333333333334, 0.596, 0.7733333333333333, 0.280434962713144, 0.6556666666666666, 0.31133333333333324, 0.651624, 0.8830309377321639)]
[ 48   2   0 150]
 
Training/ Validation / Test BACC : 0.9875 / 0.98 / 0.98  
Training/ Validation / Test Youden-index: 0.9750000000000001 / 0.96 / 0.96  
Training/ Validation / Test MCC: 0.9833321660356334 / 0.9733285267845753 / 0.9733285267845753  
Training/ Validation / Test AUC: 0.98875 / 0.98 / 0.98  
Training/ Validation / Test AUPR: 0.9962779156327544 / 0.993421052631579 / 0.993421052631579
[(0.6855, 0.7153333333333334, 0.596, 0.773333333333333

[ 49   1   0 150]
 
Training/ Validation / Test BACC : 0.9862500000000001 / 0.97 / 0.99  
Training/ Validation / Test Youden-index: 0.9725000000000001 / 0.94 / 0.98  
Training/ Validation / Test MCC: 0.9816651116404495 / 0.9599836599916586 / 0.986666070043373  
Training/ Validation / Test AUC: 0.9874999999999999 / 0.97 / 0.99  
Training/ Validation / Test AUPR: 0.9958677685950413 / 0.9901960784313726 / 0.9966887417218543
[(0.6855, 0.7153333333333334, 0.596, 0.7733333333333333, 0.280434962713144, 0.6556666666666666, 0.31133333333333324, 0.651624, 0.8830309377321639), (0.6835, 0.712, 0.598, 0.771397616468039, 0.27874226583426387, 0.655, 0.31000000000000005, 0.6489673333333333, 0.8821675985336676), (0.6865, 0.724, 0.574, 0.7759914255091104, 0.27044771175700905, 0.649, 0.29800000000000004, 0.6447766666666668, 0.8812586527151123), (0.6845, 0.712, 0.602, 0.771955186122154, 0.28217594295111026, 0.657, 0.31400000000000006, 0.6510426666666667, 0.8827596978883961), (0.6865, 0.7173333333333334, 0

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.502, 0.336, 1.0, 0.5029940119760479, 0.33511112371964275, 0.668, 0.3360000000000001, 0.668, 0.917), (0.5015, 0.3353333333333333, 1.0, 0.5022466300549177, 0.3346666719893971, 0.6676666666666666, 0.33533333333333326, 0.6676666666666666, 0.9169166666666666), (0.503, 0.3373333333333333, 1.0, 0.5044865403788634, 0.3360000424980318, 0.6686666666666666, 0.33733333333333326, 0.6686666666666666, 0.9171666666666667), (0.5025, 0.33666666666666667, 1.0, 0.5037406483790524, 0.33555558016542664, 0.6683333333333333, 0.33666666666666667, 0.6683333333333333, 0.9170833333333334), (0.5085, 0.3446666666666667, 1.0, 0.5126425384234011, 0.34088984872508615, 0.6723333333333333, 0.3446666666666667, 0.6723333333333333, 0.91808

 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5, 0.3333333333333333, 1.0, 0.5, 0.3333333333333333, 0.6666666666666666, 0.33333333333333326, 0.6520600000000001, 0.8886198901256346), (0.5, 0.3333333333333333, 1.0, 0.5, 0.3333333333333333, 0.6666666666666666, 0.33333333333333326, 0.65372, 0.888353560636172), (0.5, 0.3333333333333333, 1.0, 0.5, 0.3333333333333333, 0.6666666666666666, 0.33333333333333326, 0.6533960000000001, 0.8890413169350511), (0.5, 0.3333333333333333, 1.0, 0.5, 0.3333333333333333, 0.6666666666666666, 0.33333333333333326, 0.652244, 0.8888196842121122), (0.5, 0.3333333333333333, 1.0, 0.5, 0.3333333333333333, 0.6666666666666666, 0.33333333333333326, 0.6519, 0.8890012574054604)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Tra

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.502, 0.336, 1.0, 0.5029940119760479, 0.33511112371964275, 0.668, 0.3360000000000001, 0.7058266666666667, 0.9084059511117771), (0.505, 0.34, 1.0, 0.5074626865671642, 0.3377779740177616, 0.67, 0.3400000000000001, 0.7570226666666667, 0.9236081534504893), (0.503, 0.3373333333333333, 1.0, 0.5044865403788634, 0.3360000424980318, 0.6686666666666666, 0.33733333333333326, 0.7476893333333334, 0.9222883306035214)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.502, 0.336, 1.0

...

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9991666666666666 / 0.9966666666666666 / 1.0  
Training/ Validation / Test Youden-index: 0.9983333333333333 / 0.9933333333333332 / 1.0  
Training/ Validation / Test MCC: 0.9966777316812027 / 0.9868415319342446 / 1.0  
Training/ Validation / Test AUC: 1.0 / 0.9992 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 0.9997427626137305 / 1.0
[(0.6655, 0.6726666666666666, 0.644, 0.751023446222553, 0.27916588823184546, 0.6583333333333333, 0.31666666666666665, 0.6701719999999999, 0.9048195988831561), (0.677, 0.6606666666666666, 0.726, 0.754185692541857, 0.3376405304630877, 0.6933333333333334, 0.3866666666666667, 0.679396, 0.9017037574414568), (0.7215, 0.6693333333333333, 0.878, 0.7828460038986355, 0.47500908985311047, 0.7736666666666667, 0.5473333333333334, 0.6727466666666666, 0.9059632107724004), (0.692, 0.6666666666666666, 0.768, 0.764525993883792, 0.37899086192854936, 0.7173333333333334, 0.43466666666666676, 0.6675840000000001, 0.9038330

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9995833333333333 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9991666666666665 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9983361053365846 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.6655, 0.6726666666666666, 0.644, 0.751023446222553, 0.27916588823184546, 0.6583333333333333, 0.31666666666666665, 0.6701719999999999, 0.9048195988831561), (0.677, 0.6606666666666666, 0.726, 0.754185692541857, 0.3376405304630877, 0.6933333333333334, 0.3866666666666667, 0.679396, 0.9017037574414568), (0.7215, 0.6693333333333333, 0.878, 0.7828460038986355, 0.47500908985311047, 0.7736666666666667, 0.5473333333333334, 0.6727466666666666, 0.9059632107724004), (0.692, 0.6666666666666666, 0.768, 0.764525993883792, 0.37899086192854936, 0.7173333333333334, 0.43466666666666676, 0.6675840000000001, 0.9038330907348404), (0.6725, 0.6713333333333333, 0.676, 0.7545897339827

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [30]:
perf_raw_pbno3_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.50305,0.337400,1.0000,0.504555,0.336045,0.668700,0.337400,0.668700,0.917175
LR,0.50000,0.333333,1.0000,0.500000,0.333333,0.666667,0.333333,0.652955,0.888682
LinSVM,0.57165,0.430200,0.9960,0.600834,0.394501,0.713100,0.426200,0.686775,0.900690
MLP,0.50120,0.337400,0.9926,0.503631,0.327892,0.665000,0.330000,0.598891,0.870034
NB,0.68570,0.717000,0.5918,0.773849,0.278543,0.654400,0.308800,0.649327,0.882410
RBFSVM,0.50100,0.334667,1.0000,0.501499,0.334222,0.667333,0.334667,0.690072,0.905859
RF,0.50435,0.339133,1.0000,0.506470,0.337201,0.669567,0.339133,0.745359,0.919545


In [31]:
perf_raw_pbno3_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.81120,1.000000,0.2448,0.888306,0.437266,0.622400,0.244800,0.622400,0.899592
LR,0.54315,0.644733,0.2384,0.679127,-0.107948,0.441567,-0.116867,0.579869,0.866097
LinSVM,0.54690,0.658000,0.2136,0.685228,-0.120625,0.435800,-0.128400,0.589440,0.867672
MLP,0.68340,0.669333,0.7256,0.760577,0.345316,0.697467,0.394933,0.669253,0.903641
NB,0.86500,0.998067,0.4658,0.917293,0.622740,0.731933,0.463867,0.734550,0.924695
RBFSVM,0.72950,0.768067,0.6138,0.809812,0.352483,0.690933,0.381867,0.763404,0.919309
RF,0.92930,1.000000,0.7172,0.955018,0.809460,0.858600,0.717200,0.997991,0.999276


In [32]:
pd.concat([perf_raw_pbno3_1, perf_raw_pbno3_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.657125,0.668700,0.6224,0.696430,0.386655,0.645550,0.291100,0.645550,0.908384
LR,0.521575,0.489033,0.6192,0.589564,0.112693,0.554117,0.108233,0.616412,0.877390
LinSVM,0.559275,0.544100,0.6048,0.643031,0.136938,0.574450,0.148900,0.638108,0.884181
MLP,0.592300,0.503367,0.8591,0.632104,0.336604,0.681233,0.362467,0.634072,0.886838
NB,0.775350,0.857533,0.5288,0.845571,0.450641,0.693167,0.386333,0.691939,0.903552
RBFSVM,0.615250,0.551367,0.8069,0.655655,0.343352,0.679133,0.358267,0.726738,0.912584
RF,0.716825,0.669567,0.8586,0.730744,0.573330,0.764083,0.528167,0.871675,0.959411


In [44]:
perf_psn_pbno3_1 = get_all_clf_res(pbno3_b1_psn, pbno3_b2_psn, 'pbno3_b1b2', 'psn')
perf_psn_pbno3_2 = get_all_clf_res(pbno3_b2_psn, pbno3_b1_psn, 'pbno3_b2b1', 'psn')

[ 48   2   0 150]
 
Training/ Validation / Test BACC : 0.9770833333333333 / 0.98 / 0.98  
Training/ Validation / Test Youden-index: 0.9541666666666666 / 0.96 / 0.96  
Training/ Validation / Test MCC: 0.9682782647507626 / 0.9733285267845753 / 0.9733285267845753  
Training/ Validation / Test AUC: 0.9774666666666667 / 0.98 / 0.98  
Training/ Validation / Test AUPR: 0.9925939147627918 / 0.993421052631579 / 0.993421052631579
[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 48   2   0 150]
 
Training/ Validation / Test BACC : 0.9720833333333333 / 0.96 / 0.98  
Training/ Validation / Test Youden-index: 0.9441666666666666 / 0.9199999999999999 / 0.96  
Training/ Validation / Test MCC: 0.9615942230116677 / 0.9466276438515284 / 0.9733285267845753  
Training/ Validation / Test AUC: 0.9737114583333333 / 0.96 / 0.98  
Training/ Validation / Test AUPR: 0.9913809430645014 / 0.987012987012987 / 0.993421052631579
[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 47   3   0 150]
 
Training/ Validation / Test BACC : 0.9708333333333333 / 0.98 / 0.97  
Training/ Validation / Test Youden-index: 0.9416666666666667 / 0.96 / 0.94  
Training/ Validation / Test MCC: 0.9599228135515014 / 0.9733285267845753 / 0.9599836599916586  
Training/ Validation / Test AUC: 0.9724645833333334 / 0.98 / 0.97  
Training/ Validation / Test AUPR: 0.9909794896698184 / 0.993421052631579 / 0.9901960784313726
[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875)]


...

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9979166666666667 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9958333333333333 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9950055428500573 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99973125 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.999914916897444 / 1.0 / 1.0
[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875)]


/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[ 50   0   1 149]
 
Training/ Validation / Test BACC : 0.9995833333333333 / 1.0 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 0.9991666666666665 / 1.0 / 0.9933333333333332  
Training/ Validation / Test MCC: 0.9983361053365846 / 1.0 / 0.9868415319342446  
Training/ Validation / Test AUC: 0.9999458333333333 / 1.0 / 0.9959333333333333  
Training/ Validation / Test AUPR: 0.9999823201384959 / 1.0 / 0.9989375
[(0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0.5, 0.875), (0.75, 1.0, 0.0, 0.8571428571428571, 0.0, 0.5, 0.0, 0

/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/mnt/sypark0215/anaconda3/envs/sersnet/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [45]:
perf_psn_pbno3_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.250,0.000000,1.0000,0.000000,0.000000,0.500000,0.000000,0.500000,0.875000
LR,0.396,0.194667,1.0000,0.317595,0.233766,0.597333,0.194667,0.649279,0.886561
LinSVM,0.596,0.640000,0.4640,0.627919,0.100216,0.552000,0.104000,0.715442,0.905231
MLP,0.512,0.359800,0.9686,0.524813,0.318007,0.664200,0.328400,0.589392,0.863039
NB,0.750,1.000000,0.0000,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
RBFSVM,0.750,1.000000,0.0000,0.857143,0.000000,0.500000,0.000000,0.697291,0.910044
RF,0.250,0.000000,1.0000,0.000000,0.000000,0.500000,0.000000,0.499833,0.874927


In [46]:
perf_psn_pbno3_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.75000,1.000000,0.0,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
LR,0.57165,0.428867,1.0,0.600130,0.397602,0.714433,0.428867,0.721768,0.919798
LinSVM,0.25000,0.000000,1.0,0.000000,0.000000,0.500000,0.000000,0.500000,0.875000
MLP,0.75000,1.000000,0.0,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
NB,0.75000,1.000000,0.0,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
RBFSVM,0.75000,1.000000,0.0,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
RF,0.75000,1.000000,0.0,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000


In [47]:
pd.concat([perf_psn_pbno3_1, perf_psn_pbno3_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.500000,0.500000,0.5000,0.428571,0.000000,0.500000,0.000000,0.500000,0.875000
LR,0.483825,0.311767,1.0000,0.458862,0.315684,0.655883,0.311767,0.685524,0.903180
LinSVM,0.423000,0.320000,0.7320,0.313960,0.050108,0.526000,0.052000,0.607721,0.890116
MLP,0.631000,0.679900,0.4843,0.690978,0.159004,0.582100,0.164200,0.544696,0.869019
NB,0.750000,1.000000,0.0000,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
RBFSVM,0.750000,1.000000,0.0000,0.857143,0.000000,0.500000,0.000000,0.598646,0.892522
RF,0.500000,0.500000,0.5000,0.428571,0.000000,0.500000,0.000000,0.499917,0.874964


In [48]:
perf_bc_pbno3_1 = get_all_clf_res(pbno3_b1_bc, pbno3_b2_bc, 'pbno3_b1b2', 'bc')
perf_bc_pbno3_2 = get_all_clf_res(pbno3_b2_bc, pbno3_b1_bc, 'pbno3_b2b1', 'bc')

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9995833333333333 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9991666666666665 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9983361053365846 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.84, 0.7953333333333333, 0.974, 0.8817442719881744, 0.6808654179400332, 0.8846666666666667, 0.7693333333333334, 0.880764, 0.9661862536080636)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9995833333333333 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9991666666666665 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9983361053365846 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.84, 0.7953333333333333, 0.974, 0.8817442719881744, 0.6808654179400332, 0.8846666666666667, 0.7693333333333334, 0.880764, 0.9661862536080636), (0.834, 0.7873333333333333, 0.974, 0.87676317743

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9995833333333333 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9991666666666665 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9983361053365846 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.84, 0.7953333333333333, 0.974, 0.8817442719881744, 0.6808654179400332, 0.8846666666666667, 0.7693333333333334, 0.880764, 0.9661862536080636), (0.834, 0.7873333333333333, 0.974, 0.8767631774313289, 0.6721029355097085, 0.8806666666666667, 0.7613333333333334, 0.8802146666666666, 0.9664960071377745), (0.8355, 0.7873333333333333, 0.98, 0.8777406168710516, 0.676993586979162, 0.8836666666666666, 0.7673333333333332, 0.8810086666666667, 0.9668260048138256), (0.8355, 0.7846666666666666, 0.988, 0.8773760715616847, 0.6806430401239871, 0.8863333333333333, 0.7726666666666666, 0.8845006666666666, 0.9683637298694662), (0.836, 0.7886666666666666, 0.978, 0.8782479584261321, 0

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.562, 0.536, 0.64, 0.6473429951690821, 0.15243998463304245, 0.5880000000000001, 0.17600000000000016, 0.588, 0.8505365853658535), (0.575, 0.5866666666666667, 0.54, 0.6743295019157088, 0.11036629941195177, 0.5633333333333334, 0.1266666666666667, 0.5633333333333334, 0.8447297297297296), (0.562, 0.536, 0.64, 0.6473429951690821, 0.15243998463304245, 0.5880000000000001, 0.17600000000000016, 0.588, 0.8505365853658535), (0.562, 0.536, 0.64, 0.6473429951690821, 0.15243998463304245, 0.5880000000000001, 0.17600000000000016, 0.588, 0.8505365853658535), (0.572, 0.5506666666666666, 0.636, 0.6586921850079744, 0.1616632486798272, 0.5933333333333333, 0.18666666666666654, 0.5933333333333333, 0.8535555555555555), (0.586, 

[(0.6225, 0.512, 0.954, 0.6704495853339153, 0.41268164802374196, 0.733, 0.46599999999999997, 0.72426, 0.9102467215897956), (0.6235, 0.5126666666666667, 0.956, 0.6713225665648188, 0.41504320251887067, 0.7343333333333333, 0.46866666666666656, 0.7234413333333334, 0.9098886301602314), (0.634, 0.536, 0.928, 0.6871794871794872, 0.4070801956792859, 0.732, 0.46399999999999997, 0.7234133333333334, 0.9102368246701507), (0.6225, 0.5086666666666667, 0.964, 0.6690048224462954, 0.41952537867707274, 0.7363333333333333, 0.47266666666666657, 0.7231226666666666, 0.9101328277633147), (0.62, 0.5006666666666667, 0.978, 0.6640141467727675, 0.4268015705428588, 0.7393333333333334, 0.4786666666666668, 0.720504, 0.9099117694592345), (0.6225, 0.51, 0.96, 0.6695842450765864, 0.41677870633309155, 0.735, 0.47, 0.7227399999999999, 0.9100101788311248), (0.629, 0.52, 0.956, 0.6776715899218071, 0.42055421616452526, 0.738, 0.476, 0.7222013333333333, 0.9103353169888763), (0.622, 0.5033333333333333, 0.978, 0.6663724624889

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5865, 0.5646666666666667, 0.652, 0.6719555731852439, 0.18768022553989597, 0.6083333333333334, 0.21666666666666679, 0.698234, 0.8884094704159619), (0.581, 0.5573333333333333, 0.652, 0.6661354581673307, 0.18129704960465537, 0.6046666666666667, 0.20933333333333337, 0.690016, 0.8871659473430222), (0.582, 0.5593333333333333, 0.65, 0.6674622116149562, 0.18130575336008734, 0.6046666666666667, 0.20933333333333337, 0.6688733333333334, 0.8784400161632617), (0.593, 0.5826666666666667, 0.624, 0.682279469164715, 0.1793235752166603, 0.6033333333333333, 0.20666666666666655, 0.6318213333333333, 0.8625052803134603), (0.595, 0.5813333333333334, 0.636, 0.682850430696946, 0.1884912085744672, 0.6086666666666667, 0.21733333

...

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.682, 0.5806666666666667, 0.986, 0.7325483599663583, 0.49444115464430766, 0.7833333333333333, 0.5666666666666667, 0.9132200000000001, 0.9704152459621644), (0.6445, 0.5313333333333333, 0.984, 0.6915401301518438, 0.455026797282656, 0.7576666666666667, 0.5153333333333334, 0.9305293333333333, 0.9750156043753708), (0.706, 0.612, 0.988, 0.7574257425742574, 0.5211224233505655, 0.8, 0.6000000000000001, 0.9616846666666667, 0.9858676779096357), (0.78, 0.714, 0.978, 0.8295894655305964, 0.601314608950334, 0.846, 0.692, 0.9577453333333333, 0.9856690445123822), (0.6165, 0.49066666666666664, 0.994, 0.6574363555158553, 0.43480449084542033, 0.7423333333333333, 0.4846666666666666, 0.8833833333333332, 0.9566637260969352),

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.99875 / 0.9966666666666666 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 0.9933333333333332 / 1.0  
Training/ Validation / Test MCC: 0.9950248447588356 / 0.9868415319342446 / 1.0  
Training/ Validation / Test AUC: 0.9993395833333334 / 0.9997333333333333 / 1.0  
Training/ Validation / Test AUPR: 0.9998199935882941 / 0.999911990240502 / 1.0
[(0.635, 0.6633333333333333, 0.55, 0.7316176470588235, 0.18939221979424933, 0.6066666666666667, 0.21333333333333337, 0.6594093333333333, 0.8982889643688199), (0.6385, 0.6693333333333333, 0.546, 0.7352618088612229, 0.1916680386017295, 0.6076666666666667, 0.21533333333333338, 0.6668973333333333, 0.9009934370218095), (0.6545, 0.69, 0.548, 0.7497283592901123, 0.21351472246466996, 0.619, 0.238, 0.7002353333333333, 0.9109384876772838), (0.6245, 0.66, 0.518, 0.7250091541559868, 0.1584376665840922, 0.589, 0.17799999999999994, 0.6524706666666666, 0.8954826641513347), (0.6415, 0.6766666666666

[(0.901, 0.868, 1.0, 0.9293361884368309, 0.7885280618329282, 0.9339999999999999, 0.8679999999999999, 0.9565226666666666, 0.987352466911911), (0.9045, 0.8726666666666667, 1.0, 0.932004271982912, 0.7946395421381764, 0.9363333333333334, 0.8726666666666667, 0.9622813333333334, 0.9890680181826892), (0.897, 0.8626666666666667, 1.0, 0.9262705798138869, 0.78163529214967, 0.9313333333333333, 0.8626666666666667, 0.9527826666666667, 0.9862988462369433)]
[ 48   2   1 149]
 
Training/ Validation / Test BACC : 0.9695833333333332 / 0.9833333333333334 / 0.9766666666666666  
Training/ Validation / Test Youden-index: 0.9391666666666665 / 0.9666666666666668 / 0.9533333333333331  
Training/ Validation / Test MCC: 0.9447423380848217 / 0.9603491418152045 / 0.9598180137156622  
Training/ Validation / Test AUC: 0.9982270833333334 / 0.9990666666666667 / 0.9993333333333333  
Training/ Validation / Test AUPR: 0.9994129078228344 / 0.9996886585284666 / 0.9997773045069666
[(0.901, 0.868, 1.0, 0.9293361884368309, 0.

[ 45   5   0 150]
 
Training/ Validation / Test BACC : 0.9633333333333334 / 0.9366666666666666 / 0.95  
Training/ Validation / Test Youden-index: 0.9266666666666667 / 0.8733333333333333 / 0.8999999999999999  
Training/ Validation / Test MCC: 0.9362180009270741 / 0.9056044800645714 / 0.9332565252573827  
Training/ Validation / Test AUC: 0.99806875 / 0.9973333333333334 / 0.9986666666666666  
Training/ Validation / Test AUPR: 0.9993594373668588 / 0.9991231724638407 / 0.9995540242186118
[(0.901, 0.868, 1.0, 0.9293361884368309, 0.7885280618329282, 0.9339999999999999, 0.8679999999999999, 0.9565226666666666, 0.987352466911911), (0.9045, 0.8726666666666667, 1.0, 0.932004271982912, 0.7946395421381764, 0.9363333333333334, 0.8726666666666667, 0.9622813333333334, 0.9890680181826892), (0.897, 0.8626666666666667, 1.0, 0.9262705798138869, 0.78163529214967, 0.9313333333333333, 0.8626666666666667, 0.9527826666666667, 0.9862988462369433), (0.9005, 0.8673333333333333, 1.0, 0.9289539450196358, 0.787661164

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.9933333333333332  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.9868415319342446  
Training/ Validation / Test AUC: 1.0 / 1.0 / 0.9994666666666667  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 0.999826273943921
[(0.865, 0.8206666666666667, 0.998, 0.9011713030746705, 0.7288728536064344, 0.9093333333333333, 0.8186666666666667, 0.979492, 0.9936277736980319), (0.702, 0.6053333333333333, 0.992, 0.7529021558872305, 0.5193205584696801, 0.7986666666666666, 0.5973333333333333, 0.8247173333333333, 0.9485942050133226), (0.7255, 0.6393333333333333, 0.984, 0.7774625050668829, 0.5401166753326083, 0.8116666666666666, 0.6233333333333333, 0.8418760000000001, 0.9540185217989778), (0.8405, 0.788, 0.998, 0.8811032426388371, 0.6923883902555485, 0.893, 0.786, 0.9778866666666666, 0.9931600553109756), (0.846, 0.7953333333333333, 0.998, 0.8856718634001485, 0.700352

In [49]:
perf_bc_pbno3_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.5707,0.561067,0.5996,0.661987,0.139397,0.580333,0.160667,0.580333,0.849305
LR,0.6235,0.511467,0.9596,0.670746,0.417622,0.735533,0.471067,0.722502,0.909989
LinSVM,0.6443,0.630267,0.6864,0.725059,0.278524,0.658333,0.316667,0.734840,0.911973
MLP,0.6442,0.531600,0.9820,0.690925,0.453931,0.756800,0.513600,0.679682,0.904191
NB,0.8356,0.787867,0.9788,0.877875,0.676614,0.883333,0.766667,0.881102,0.966853
RBFSVM,0.6356,0.514133,1.0000,0.679085,0.457400,0.757067,0.514133,0.805747,0.937770
RF,0.5888,0.573400,0.6350,0.676500,0.180685,0.604200,0.208400,0.674736,0.882808


In [50]:
perf_bc_pbno3_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.84630,0.897600,0.6924,0.896472,0.597264,0.795000,0.590000,0.795000,0.935987
LR,0.60995,0.643333,0.5098,0.712139,0.135648,0.576567,0.153133,0.658693,0.894230
LinSVM,0.63905,0.674333,0.5332,0.736982,0.185389,0.603767,0.207533,0.673528,0.902482
MLP,0.79635,0.729867,0.9958,0.840413,0.637124,0.862833,0.725667,0.924856,0.977869
NB,0.84885,0.998667,0.3994,0.908350,0.572351,0.699033,0.398067,0.704268,0.917556
RBFSVM,0.90065,0.867533,1.0000,0.929061,0.787965,0.933767,0.867533,0.959456,0.988227
RF,0.68725,0.588600,0.9832,0.735671,0.500750,0.785900,0.571800,0.935430,0.977020


In [51]:
pd.concat([perf_bc_pbno3_1, perf_bc_pbno3_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.708500,0.729333,0.6460,0.779229,0.368331,0.687667,0.375333,0.687667,0.892646
LR,0.616725,0.577400,0.7347,0.691442,0.276635,0.656050,0.312100,0.690598,0.902110
LinSVM,0.641675,0.652300,0.6098,0.731020,0.231956,0.631050,0.262100,0.704184,0.907227
MLP,0.720275,0.630733,0.9889,0.765669,0.545527,0.809817,0.619633,0.802269,0.941030
NB,0.842225,0.893267,0.6891,0.893112,0.624482,0.791183,0.582367,0.792685,0.942205
RBFSVM,0.768125,0.690833,1.0000,0.804073,0.622683,0.845417,0.690833,0.882602,0.962998
RF,0.638025,0.581000,0.8091,0.706086,0.340717,0.695050,0.390100,0.805083,0.929914


In [52]:
perf_bn_pbno3_1 = get_all_clf_res(pbno3_b1_bn, pbno3_b2_bn, 'pbno3_b1b2', 'bn')
perf_bn_pbno3_2 = get_all_clf_res(pbno3_b2_bn, pbno3_b1_bn, 'pbno3_b2b1', 'bn')

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.98875 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 0.9775 / 0.98 / 1.0  
Training/ Validation / Test MCC: 0.9849991497773156 / 0.986666070043373 / 1.0  
Training/ Validation / Test AUC: 0.98875 / 0.99 / 1.0  
Training/ Validation / Test AUPR: 0.9962779156327544 / 0.9966887417218543 / 1.0
[(0.36, 0.4746666666666667, 0.016, 0.5266272189349113, -0.45057070742808963, 0.24533333333333335, -0.5093333333333333, 0.24381, 0.72995084774928)]
[ 48   2   0 150]
 
Training/ Validation / Test BACC : 0.99 / 1.0 / 0.98  
Training/ Validation / Test Youden-index: 0.98 / 1.0 / 0.96  
Training/ Validation / Test MCC: 0.986666070043373 / 1.0 / 0.9733285267845753  
Training/ Validation / Test AUC: 0.99 / 1.0 / 0.98  
Training/ Validation / Test AUPR: 0.9966887417218543 / 1.0 / 0.993421052631579
[(0.36, 0.4746666666666667, 0.016, 0.5266272189349113, -0.45057070742808963, 0.24533333333333335, -0.5093333333333333, 0.24381, 0.72995084774928),

[ 49   1   0 150]
 
Training/ Validation / Test BACC : 0.99 / 0.99 / 0.99  
Training/ Validation / Test Youden-index: 0.98 / 0.98 / 0.98  
Training/ Validation / Test MCC: 0.986666070043373 / 0.986666070043373 / 0.986666070043373  
Training/ Validation / Test AUC: 0.99 / 0.99 / 0.99  
Training/ Validation / Test AUPR: 0.9966887417218543 / 0.9966887417218543 / 0.9966887417218543
[(0.36, 0.4746666666666667, 0.016, 0.5266272189349113, -0.45057070742808963, 0.24533333333333335, -0.5093333333333333, 0.24381, 0.72995084774928), (0.3605, 0.47533333333333333, 0.016, 0.5271719038817005, -0.4500770028829032, 0.24566666666666667, -0.5086666666666666, 0.24413066666666666, 0.7301319257126073), (0.3605, 0.47533333333333333, 0.016, 0.5271719038817005, -0.4500770028829032, 0.24566666666666667, -0.5086666666666666, 0.24362399999999998, 0.7301096252509893), (0.36, 0.4746666666666667, 0.016, 0.5266272189349113, -0.45057070742808963, 0.24533333333333335, -0.5093333333333333, 0.24380333333333332, 0.7299492

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.3005, 0.386, 0.044, 0.45287446226046146, -0.4944383487281012, 0.215, -0.5700000000000001, 0.21500000000000002, 0.6971383632923368), (0.616, 0.5673333333333334, 0.762, 0.6890688259109312, 0.28533946464239307, 0.6646666666666667, 0.3293333333333335, 0.6646666666666667, 0.8845764604810997), (0.299, 0.3873333333333333, 0.034, 0.453198127925117, -0.5021695321466091, 0.21066666666666667, -0.5786666666666667, 0.21066666666666667, 0.6964429824561403), (0.472, 0.5306666666666666, 0.296, 0.6012084592145015, -0.15178259695273458, 0.41333333333333333, -0.17333333333333334, 0.41333333333333333, 0.7880232288037166), (0.355, 0.454, 0.058, 0.5135746606334841, -0.42758876749518376, 0.256, -0.488, 0.256, 0.7273229166666

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.3815, 0.4513333333333333, 0.172, 0.5225781551524508, -0.32756199202526826, 0.31166666666666665, -0.3766666666666667, 0.3706, 0.7672296280176912), (0.381, 0.452, 0.168, 0.5227447956823439, -0.33055327814163704, 0.31, -0.38, 0.3706593333333334, 0.7670002015326073), (0.382, 0.4513333333333333, 0.174, 0.5227799227799228, -0.3257929957033038, 0.31266666666666665, -0.3746666666666667, 0.37088, 0.7672083628018351), (0.3895, 0.454, 0.196, 0.5272938443670151, -0.30415837544649904, 0.325, -0.35, 0.3751446666666667, 0.7684028015366613), (0.383, 0.454, 0.17, 0.524653312788906, -0.32713648600062584, 0.312, -0.376, 0.3719633333333333, 0.767575446746146)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.3765, 0.46, 0.126, 0.525314046440807, -0.36146137218948327, 0.29300000000000004, -0.4139999999999999, 0.37722933333333336, 0.7722086428289372), (0.3725, 0.4613333333333333, 0.106, 0.5244410761652141, -0.3783734297745664, 0.2836666666666667, -0.43266666666666664, 0.3755793333333334, 0.7731767118202761)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.3765, 0.46, 0.126, 0.525314046440807, -0.36146137218948327, 0.29300000000000004, -0.4139999999999999, 0.37722933333333

...

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 1.0 / 0.9966666666666666 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 1.0 / 0.9933333333333332 / 0.9933333333333332  
Training/ Validation / Test MCC: 1.0 / 0.9868415319342446 / 0.9868415319342446  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.233, 0.31066666666666665, 0.0, 0.37793998377939986, -0.5973255323359284, 0.15533333333333332, -0.6893333333333334, 0.01664333333333333, 0.5400463227279766), (0.2065, 0.2753333333333333, 0.0, 0.3423124740986324, -0.6299683864643568, 0.13766666666666666, -0.7246666666666667, 0.01279533333333333, 0.5393977878761533), (0.24, 0.32, 0.0, 0.3870967741935484, -0.5890150893739515, 0.16, -0.6799999999999999, 0.01095266666666667, 0.5391186486661422), (0.2265, 0.302, 0.0, 0.36934366082348147, -0.6051544945090915, 0.151, -0.698, 0.011332666666666668, 0.5391235432556233), (0.238, 0.31733333333333336, 0.0, 0.38449111470113084, -

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.998333332174959 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.7255, 0.9673333333333334, 0.0, 0.8409156766154737, -0.09149740519839136, 0.4836666666666667, -0.03266666666666662, 0.4069813333333334, 0.782121337059703), (0.699, 0.932, 0.0, 0.8228369629193644, -0.1338416723448232, 0.466, -0.06799999999999995, 0.4188186666666666, 0.7852202397093924), (0.609, 0.812, 0.0, 0.7569919204474829, -0.23391191639866177, 0.406, -0.18799999999999994, 0.49722666666666665, 0.7860375006841178), (0.6175, 0.8233333333333334, 0.0, 0.7635239567233385, -0.22563817064892727, 0.4116666666666667, -0.17666666666666664, 0.543528, 0.8206269819019061)]
[ 49   1   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 1.0 / 

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9975 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 0.9950000000000001 / 0.98 / 1.0  
Training/ Validation / Test MCC: 0.9966666573918979 / 0.986666070043373 / 1.0  
Training/ Validation / Test AUC: 0.9999875 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9999958333260947 / 1.0 / 1.0
[(0.4825, 0.6433333333333333, 0.0, 0.6509274873524452, -0.348897173634573, 0.32166666666666666, -0.3566666666666667, 0.140384, 0.5788699751665505)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9975 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9950000000000001 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9966666573918979 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99999375 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.999997918110777 / 1.0 / 1.0
[(0.4825, 0.6433333333333333, 0.0, 0.6509274873524452, -0.348897173634573, 0.32166666666666666, -0.3566666666666667, 0.140384, 0.5788699751665505), (0.475, 0.

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 0.9970833333333333 / 1.0 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 0.9941666666666666 / 1.0 / 0.9933333333333332  
Training/ Validation / Test MCC: 0.9949972141058697 / 1.0 / 0.9868415319342446  
Training/ Validation / Test AUC: 0.9999958333333333 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9999986128448146 / 1.0 / 1.0
[(0.4825, 0.6433333333333333, 0.0, 0.6509274873524452, -0.348897173634573, 0.32166666666666666, -0.3566666666666667, 0.140384, 0.5788699751665505), (0.475, 0.6333333333333333, 0.0, 0.6440677966101694, -0.35557950110937975, 0.31666666666666665, -0.3666666666666667, 0.15840933333333335, 0.5870962180566374), (0.4735, 0.6313333333333333, 0.0, 0.6426874787919918, -0.35691736060833, 0.31566666666666665, -0.3686666666666667, 0.15126533333333333, 0.5824074217389111), (0.48, 0.64, 0.0, 0.6486486486486487, -0.35112344158839165, 0.32, -0.36, 0.17496933333333334, 0.5994133993934592), (0.4795, 0.63933

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 0.9966666666666666 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 0.9933333333333332 / 1.0  
Training/ Validation / Test MCC: 1.0 / 0.9868415319342446 / 1.0  
Training/ Validation / Test AUC: 1.0 / 0.9998666666666667 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 0.9999557027225903 / 1.0
[(0.5095, 0.6793333333333333, 0.0, 0.6750579662139782, -0.32488751619387635, 0.3396666666666667, -0.32066666666666666, 0.03781466666666668, 0.5430258247180817), (0.532, 0.7093333333333334, 0.0, 0.6945169712793734, -0.30483458821499904, 0.3546666666666667, -0.29066666666666663, 0.047997333333333336, 0.5454466117704782), (0.529, 0.7053333333333334, 0.0, 0.6919555264879006, -0.3075153370141565, 0.3526666666666667, -0.29466666666666663, 0.02829600000000001, 0.5418171619907255), (0.5315, 0.7086666666666667, 0.0, 0.6940907606921319, -0.30528158096532804, 0.35433333333333333, -0.29133333333333333, 0.06348000000000001, 0.5490181741018514), 

In [53]:
perf_bn_pbno3_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.41260,0.469200,0.2428,0.546425,-0.251494,0.356000,-0.288000,0.356000,0.761580
LR,0.38360,0.452867,0.1758,0.524273,-0.322941,0.314333,-0.371333,0.372015,0.767632
LinSVM,0.38070,0.438733,0.2066,0.515185,-0.307625,0.322667,-0.354667,0.365352,0.765573
MLP,0.45635,0.526800,0.2450,0.592404,-0.200471,0.385900,-0.228200,0.401129,0.769605
NB,0.36050,0.475600,0.0152,0.527311,-0.450629,0.245400,-0.509200,0.243762,0.730134
RBFSVM,0.38085,0.467933,0.1196,0.531319,-0.360873,0.293767,-0.412467,0.277710,0.675654
RF,0.36780,0.456333,0.1022,0.519853,-0.385778,0.279267,-0.441467,0.373012,0.771893


In [54]:
perf_bn_pbno3_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.42700,0.495600,0.2212,0.476158,-0.262031,0.358400,-0.283200,0.358400,0.777429
LR,0.62770,0.836933,0.0000,0.770549,-0.212930,0.418467,-0.163067,0.432203,0.766036
LinSVM,0.65115,0.868200,0.0000,0.787890,-0.186881,0.434100,-0.131800,0.471923,0.790373
MLP,0.52285,0.697133,0.0000,0.686596,-0.312989,0.348567,-0.302867,0.052168,0.548380
NB,0.38560,0.514133,0.0000,0.555428,-0.437993,0.257067,-0.485867,0.251167,0.743588
RBFSVM,0.47680,0.635733,0.0000,0.645711,-0.353976,0.317867,-0.364267,0.156843,0.587472
RF,0.28055,0.374067,0.0000,0.420851,-0.547669,0.187033,-0.625933,0.047905,0.566993


In [55]:
pd.concat([perf_bn_pbno3_1, perf_bn_pbno3_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.419800,0.482400,0.2320,0.511291,-0.256763,0.357200,-0.285600,0.357200,0.769505
LR,0.505650,0.644900,0.0879,0.647411,-0.267935,0.366400,-0.267200,0.402109,0.766834
LinSVM,0.515925,0.653467,0.1033,0.651537,-0.247253,0.378383,-0.243233,0.418637,0.777973
MLP,0.489600,0.611967,0.1225,0.639500,-0.256730,0.367233,-0.265533,0.226648,0.658992
NB,0.373050,0.494867,0.0076,0.541369,-0.444311,0.251233,-0.497533,0.247464,0.736861
RBFSVM,0.428825,0.551833,0.0598,0.588515,-0.357425,0.305817,-0.388367,0.217276,0.631563
RF,0.324175,0.415200,0.0511,0.470352,-0.466724,0.233150,-0.533700,0.210458,0.669443


In [56]:
perf_bc_psn_pbno3_1 = get_all_clf_res(pbno3_b1_bl_psn, pbno3_b2_bl_psn, 'pbno3_b1b2', 'bc_psn')
perf_bc_psn_pbno3_2 = get_all_clf_res(pbno3_b2_bl_psn, pbno3_b1_bl_psn, 'pbno3_b2b1', 'bc_psn')

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9950248447588356 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.7565, 0.882, 0.38, 0.8445579316948612, 0.29309323775921514, 0.631, 0.262, 0.6316246666666667, 0.8902586984601749)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.99875 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9975 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9950248447588356 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.7565, 0.882, 0.38, 0.8445579316948612, 0.29309323775921514, 0.631, 0.262, 0.6316246666666667, 0.8902586984601749), (0.758, 0.8746666666666667, 0.408, 0.8442728442728443, 0.3083324071849608, 0.6413333333333333, 0.2826666666666666, 0.6438526666666666, 0.89258885567

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 0.9991666666666666 / 1.0 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 0.9983333333333333 / 1.0 / 0.9933333333333332  
Training/ Validation / Test MCC: 0.9966777316812027 / 1.0 / 0.9868415319342446  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.7565, 0.882, 0.38, 0.8445579316948612, 0.29309323775921514, 0.631, 0.262, 0.6316246666666667, 0.8902586984601749), (0.758, 0.8746666666666667, 0.408, 0.8442728442728443, 0.3083324071849608, 0.6413333333333333, 0.2826666666666666, 0.6438526666666666, 0.8925888556748789), (0.7545, 0.8773333333333333, 0.386, 0.8427793788024336, 0.2915456280655236, 0.6316666666666666, 0.2633333333333332, 0.6329373333333335, 0.8901514157745419), (0.759, 0.88, 0.396, 0.8456117873158232, 0.3052589379531447, 0.638, 0.276, 0.6374466666666667, 0.8915746848365073), (0.757, 0.8813333333333333, 0.384, 0.8447284345047923, 0.29588829170738246, 0

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 0.98 / 1.0  
Training/ Validation / Test MCC: 1.0 / 0.986666070043373 / 1.0  
Training/ Validation / Test AUC: 1.0 / 0.99 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 0.9966887417218543 / 1.0
[(0.596, 0.6646666666666666, 0.39, 0.7116345467523197, 0.049661521559370286, 0.5273333333333333, 0.05466666666666664, 0.5273333333333333, 0.8409558371735792), (0.613, 0.714, 0.31, 0.7345679012345679, 0.022856189240115815, 0.512, 0.02400000000000002, 0.512, 0.8424279661016949), (0.6015, 0.7133333333333334, 0.266, 0.7286346612189308, -0.019898415033961017, 0.4896666666666667, -0.02066666666666661, 0.4896666666666667, 0.8364700765483646), (0.62, 0.7393333333333333, 0.262, 0.7447951645399598, 0.0013146099037937037, 0.5006666666666666, 0.0013333333333331865, 0.5006666666666666, 0.8425858141632837), (0.611, 0.7293333333333333, 0.256, 0.7376938637896157, -0.01435570217376662, 0.492

[(0.6105, 0.482, 0.996, 0.6498876404494383, 0.430560745750159, 0.739, 0.478, 0.7422733333333333, 0.9136017774450906), (0.6085, 0.4806666666666667, 0.992, 0.6480898876404494, 0.4257567206929745, 0.7363333333333333, 0.47266666666666657, 0.7435906666666666, 0.913622280199901), (0.61, 0.4806666666666667, 0.998, 0.648964896489649, 0.4315486786457612, 0.7393333333333334, 0.4786666666666668, 0.7425266666666667, 0.9140337233554934), (0.6105, 0.48333333333333334, 0.992, 0.6505159264244056, 0.4276538272436584, 0.7376666666666667, 0.4753333333333334, 0.7432733333333332, 0.9138751745027813), (0.612, 0.48733333333333334, 0.986, 0.6532618409294012, 0.4247563331888938, 0.7366666666666667, 0.4733333333333334, 0.743308, 0.9129543601253746), (0.6125, 0.488, 0.986, 0.6538633318445735, 0.42523520356409056, 0.737, 0.474, 0.741556, 0.912253775898012), (0.6125, 0.4846666666666667, 0.996, 0.6523104531179902, 0.43245218715663075, 0.7403333333333333, 0.4806666666666666, 0.7422306666666667, 0.9138864039648273), 

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 0.99 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 0.98 / 1.0  
Training/ Validation / Test MCC: 1.0 / 0.986666070043373 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.6255, 0.7046666666666667, 0.388, 0.7383863080684596, 0.08612644646520973, 0.5463333333333333, 0.09266666666666667, 0.6295953333333333, 0.8718287100880087), (0.6145, 0.6406666666666667, 0.536, 0.7137021908652061, 0.15592948961911368, 0.5883333333333334, 0.17666666666666675, 0.6331726666666667, 0.8735509650726145), (0.6395, 0.6866666666666666, 0.498, 0.7407407407407407, 0.1666403356255983, 0.5923333333333334, 0.18466666666666676, 0.6337233333333332, 0.8718210559794153), (0.6015, 0.6233333333333333, 0.536, 0.7011623547056618, 0.1399520759485274, 0.5796666666666667, 0.15933333333333333, 0.6351926666666666, 0.876635279892501), (0.623, 0.7166666666666667, 0.342, 0.740358126721763, 0.055541213

...

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.9933333333333332  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.9868415319342446  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.6725, 0.8853333333333333, 0.034, 0.8021745696164301, -0.11940832758247752, 0.45966666666666667, -0.08066666666666666, 0.616788, 0.8805371773657922), (0.6425, 0.8333333333333334, 0.07, 0.7776049766718507, -0.119743757696188, 0.45166666666666666, -0.09666666666666668, 0.63664, 0.886446762081055), (0.5235, 0.6073333333333333, 0.272, 0.6565765765765765, -0.10869106691879886, 0.43966666666666665, -0.1206666666666667, 0.5996319999999999, 0.8764779018612068), (0.6235, 0.756, 0.226, 0.7507447864945382, -0.01826294339588071, 0.491, -0.018000000000000016, 0.6600253333333332, 0.8912320036207917), (0.585, 0.7413333333333333, 0.116, 0.7282252783235101, -0.148408989967775

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9991666666666666 / 0.9966666666666666 / 1.0  
Training/ Validation / Test Youden-index: 0.9983333333333333 / 0.9933333333333332 / 1.0  
Training/ Validation / Test MCC: 0.9966777316812027 / 0.9868415319342446 / 1.0  
Training/ Validation / Test AUC: 0.9994124999999999 / 0.9985333333333334 / 1.0  
Training/ Validation / Test AUPR: 0.9998409007329405 / 0.9995430900621116 / 1.0
[(0.5945, 0.4593333333333333, 1.0, 0.6295111923252626, 0.4185502809170405, 0.7296666666666667, 0.45933333333333337, 0.6187053333333334, 0.8824802753055321), (0.6085, 0.478, 1.0, 0.6468200270635994, 0.4316041603229816, 0.739, 0.478, 0.6252626666666667, 0.8853398737940369), (0.605, 0.4746666666666667, 0.996, 0.6431797651309846, 0.4253774696287899, 0.7353333333333334, 0.4706666666666668, 0.6163813333333332, 0.8818749107078901), (0.6185, 0.49133333333333334, 1.0, 0.6589181940098346, 0.4410333603189507, 0.7456666666666667, 0.4913333333333334, 0.6379666666666666, 0

[(0.925, 0.912, 0.964, 0.9480249480249481, 0.8223731778969653, 0.938, 0.8759999999999999, 0.9569146666666666, 0.9877886996541166), (0.925, 0.9153333333333333, 0.954, 0.9482044198895028, 0.8198903702523263, 0.9346666666666666, 0.8693333333333333, 0.9574360000000001, 0.9879368585954956), (0.928, 0.9233333333333333, 0.942, 0.9505833905284832, 0.8232667060844615, 0.9326666666666666, 0.8653333333333333, 0.9598353333333334, 0.9886462895436273)]
[ 50   0   1 149]
 
Training/ Validation / Test BACC : 0.9924999999999999 / 0.99 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 0.9849999999999999 / 0.98 / 0.9933333333333332  
Training/ Validation / Test MCC: 0.9833665098891332 / 0.986666070043373 / 0.9868415319342446  
Training/ Validation / Test AUC: 0.99984375 / 0.9998666666666667 / 0.9998666666666667  
Training/ Validation / Test AUPR: 0.999948394499887 / 0.9999557027225902 / 0.9999557027225902
[(0.925, 0.912, 0.964, 0.9480249480249481, 0.8223731778969653, 0.938, 0.8759999999999

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.9933333333333334 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9866666666666668 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9866666666666667 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.9998916666666666 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9999642251263323 / 1.0 / 1.0
[(0.925, 0.912, 0.964, 0.9480249480249481, 0.8223731778969653, 0.938, 0.8759999999999999, 0.9569146666666666, 0.9877886996541166), (0.925, 0.9153333333333333, 0.954, 0.9482044198895028, 0.8198903702523263, 0.9346666666666666, 0.8693333333333333, 0.9574360000000001, 0.9879368585954956), (0.928, 0.9233333333333333, 0.942, 0.9505833905284832, 0.8232667060844615, 0.9326666666666666, 0.8653333333333333, 0.9598353333333334, 0.9886462895436273), (0.931, 0.922, 0.958, 0.9524793388429752, 0.8331181962874523, 0.94, 0.8799999999999999, 0.9617173333333333, 0.989131605521699), (0.9295, 0.9153333333333333, 0.972, 0.9511603740907516, 0.83339129595

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.9933333333333332  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.9868415319342446  
Training/ Validation / Test AUC: 1.0 / 1.0 / 0.9995999999999999  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 0.9998688510491915
[(0.6805, 0.8686666666666667, 0.116, 0.8030816640986133, -0.019906706015753214, 0.49233333333333335, -0.01533333333333331, 0.8024533333333334, 0.9420405451004394), (0.692, 0.8913333333333333, 0.094, 0.8127659574468085, -0.020716953468976475, 0.49266666666666664, -0.014666666666666717, 0.8133813333333333, 0.9452532457614115), (0.7245, 0.934, 0.096, 0.8356695496570236, 0.04978007073478734, 0.515, 0.030000000000000027, 0.8698986666666667, 0.961991254937266), (0.758, 0.9753333333333334, 0.106, 0.8580645161290322, 0.16988743721203844, 0.5406666666666667, 0.08133333333333348, 0.9443719999999999, 0.9837154998693034), (0.7495, 0.968, 0.094,

In [57]:
perf_bc_psn_pbno3_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.60915,0.725400,0.2604,0.735377,-0.016812,0.492900,-0.014200,0.492900,0.839165
LR,0.61095,0.483867,0.9922,0.651025,0.428236,0.738033,0.476067,0.742630,0.913443
LinSVM,0.60860,0.482200,0.9878,0.648572,0.423510,0.735000,0.470000,0.749698,0.917601
MLP,0.51195,0.349267,1.0000,0.517617,0.343962,0.674633,0.349267,0.701085,0.910372
NB,0.75735,0.880533,0.3878,0.844797,0.298263,0.634167,0.268333,0.634456,0.890801
RBFSVM,0.66000,0.546667,1.0000,0.706865,0.481311,0.773333,0.546667,0.898988,0.966597
RF,0.62105,0.663400,0.4940,0.723581,0.140610,0.578700,0.157400,0.635898,0.874529


In [58]:
perf_bc_psn_pbno3_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.83075,0.799333,0.9250,0.871293,0.660649,0.862167,0.724333,0.862167,0.959222
LR,0.66950,0.641333,0.7540,0.744049,0.343984,0.697667,0.395333,0.744045,0.918601
LinSVM,0.61060,0.481000,0.9994,0.649122,0.433294,0.740200,0.480400,0.628021,0.886348
MLP,0.72765,0.937600,0.0978,0.837285,0.078985,0.517700,0.035400,0.883008,0.965605
NB,0.75000,1.000000,0.0000,0.857143,0.000000,0.500000,0.000000,0.500000,0.875000
RBFSVM,0.92645,0.916133,0.9574,0.949195,0.823719,0.936767,0.873533,0.957776,0.988040
RF,0.63175,0.752200,0.2704,0.750906,0.014865,0.511300,0.022600,0.660846,0.892370


In [59]:
pd.concat([perf_bc_psn_pbno3_1, perf_bc_psn_pbno3_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.719950,0.762367,0.5927,0.803335,0.321919,0.677533,0.355067,0.677533,0.899193
LR,0.640225,0.562600,0.8731,0.697537,0.386110,0.717850,0.435700,0.743338,0.916022
LinSVM,0.609600,0.481600,0.9936,0.648847,0.428402,0.737600,0.475200,0.688859,0.901974
MLP,0.619800,0.643433,0.5489,0.677451,0.211474,0.596167,0.192333,0.792047,0.937988
NB,0.753675,0.940267,0.1939,0.850970,0.149132,0.567083,0.134167,0.567228,0.882900
RBFSVM,0.793225,0.731400,0.9787,0.828030,0.652515,0.855050,0.710100,0.928382,0.977318
RF,0.626400,0.707800,0.3822,0.737244,0.077737,0.545000,0.090000,0.648372,0.883449


In [60]:
perf_bc_bn_pbno3_1 = get_all_clf_res(pbno3_b1_bl_bn, pbno3_b2_bl_bn, 'pbno3_b1b2', 'bc_bn')
perf_bc_bn_pbno3_2 = get_all_clf_res(pbno3_b2_bl_bn, pbno3_b1_bl_bn, 'pbno3_b2b1', 'bc_bn')

[ 49   1   0 150]
 
Training/ Validation / Test BACC : 0.995 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 0.99 / 1.0 / 0.98  
Training/ Validation / Test MCC: 0.9933332590098599 / 1.0 / 0.986666070043373  
Training/ Validation / Test AUC: 0.995 / 1.0 / 0.99  
Training/ Validation / Test AUPR: 0.9983388704318936 / 1.0 / 0.9966887417218543
[(0.6905, 0.9206666666666666, 0.0, 0.8169180715764567, -0.14521733271084644, 0.4603333333333333, -0.07933333333333337, 0.45966666666666667, 0.8568752542002259)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.995 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.99 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9933332590098599 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.995 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9983388704318936 / 1.0 / 1.0
[(0.6905, 0.9206666666666666, 0.0, 0.8169180715764567, -0.14521733271084644, 0.4603333333333333, -0.07933333333333337, 0.45966666666666667, 0.8568752542002259), (0.688

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 0.99375 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 0.9875 / 1.0 / 1.0  
Training/ Validation / Test MCC: 0.9916665213803257 / 1.0 / 1.0  
Training/ Validation / Test AUC: 0.99375 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 0.9979253112033195 / 1.0 / 1.0
[(0.6905, 0.9206666666666666, 0.0, 0.8169180715764567, -0.14521733271084644, 0.4603333333333333, -0.07933333333333337, 0.45966666666666667, 0.8568752542002259), (0.688, 0.9173333333333333, 0.0, 0.8151658767772512, -0.1484341391812703, 0.45866666666666667, -0.08266666666666667, 0.458, 0.8561044245533587), (0.6825, 0.91, 0.0, 0.8112927191679049, -0.15533411192354932, 0.455, -0.08999999999999997, 0.454, 0.8542490301081922), (0.6855, 0.914, 0.0, 0.8134084841293385, -0.1515993481947153, 0.457, -0.08599999999999997, 0.45666666666666667, 0.8554872925967673), (0.6915, 0.922, 0.0, 0.8176174992610109, -0.14391535837796426, 0.461, -0.07799999999999996, 0.46066666666666667, 0.8

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.304, 0.404, 0.004, 0.46543778801843316, -0.5154823466322014, 0.20400000000000001, -0.592, 0.20400000000000001, 0.6999565217391304), (0.305, 0.406, 0.002, 0.46702453987730064, -0.5157034431545978, 0.20400000000000001, -0.592, 0.20400000000000001, 0.7005694945848375), (0.305, 0.406, 0.002, 0.46702453987730064, -0.5157034431545978, 0.20400000000000001, -0.592, 0.20400000000000001, 0.7005694945848375), (0.302, 0.4013333333333333, 0.004, 0.46307692307692305, -0.5175908970416082, 0.20266666666666666, -0.5946666666666667, 0.20266666666666666, 0.6988030303030304), (0.305, 0.406, 0.002, 0.46702453987730064, -0.5157034431545978, 0.20400000000000001, -0.592, 0.20400000000000001, 0.7005694945848375), (0.3055, 0.40

[(0.5185, 0.6666666666666666, 0.074, 0.6749915626054674, -0.2533842802142521, 0.3703333333333333, -0.2593333333333334, 0.5217413333333334, 0.8315923415765399), (0.527, 0.6646666666666666, 0.114, 0.6782312925170068, -0.21345296744756048, 0.3893333333333333, -0.22133333333333338, 0.5556613333333333, 0.851406095987818), (0.53, 0.666, 0.122, 0.6800544588155207, -0.20422969643517394, 0.394, -0.21199999999999997, 0.5691426666666667, 0.8569444030482383), (0.518, 0.6666666666666666, 0.072, 0.6747638326585695, -0.2554891764824422, 0.3693333333333333, -0.2613333333333334, 0.481996, 0.81098323368571), (0.53, 0.666, 0.122, 0.6800544588155207, -0.20422969643517394, 0.394, -0.21199999999999997, 0.5617093333333334, 0.853311383039064), (0.531, 0.666, 0.126, 0.6805177111716622, -0.20015995500120465, 0.396, -0.20799999999999996, 0.5670546666666667, 0.8556916817321656), (0.5185, 0.6666666666666666, 0.074, 0.6749915626054674, -0.2533842802142521, 0.3703333333333333, -0.2593333333333334, 0.5224746666666666

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.491, 0.6206666666666667, 0.102, 0.6465277777777778, -0.25965537390802124, 0.36133333333333334, -0.2773333333333333, 0.30767533333333336, 0.6816074471406088), (0.4775, 0.6266666666666667, 0.03, 0.6427350427350428, -0.3284772129546587, 0.32833333333333337, -0.34333333333333327, 0.25845066666666666, 0.6635862950562667), (0.428, 0.5226666666666666, 0.144, 0.5781710914454278, -0.2953894351280011, 0.3333333333333333, -0.33333333333333337, 0.2983226666666667, 0.6769862055013804), (0.4965, 0.648, 0.042, 0.658759742460183, -0.30079627039276047, 0.34500000000000003, -0.30999999999999994, 0.262994, 0.6641796965688205), (0.479, 0.596, 0.128, 0.6318021201413427, -0.253207457205044, 0.362, -0.276, 0.3366246666666667

...

[ 50   0   1 149]
 
Training/ Validation / Test BACC : 1.0 / 0.9766666666666666 / 0.9966666666666666  
Training/ Validation / Test Youden-index: 1.0 / 0.9533333333333331 / 0.9933333333333332  
Training/ Validation / Test MCC: 1.0 / 0.9598180137156622 / 0.9868415319342446  
Training/ Validation / Test AUC: 1.0 / 0.9985333333333333 / 0.9995999999999999  
Training/ Validation / Test AUPR: 1.0 / 0.9995040461613744 / 0.9998688510491915
[(0.527, 0.7026666666666667, 0.0, 0.6902423051735429, -0.30930097909171844, 0.35133333333333333, -0.29733333333333334, 0.18025666666666665, 0.6432389449045198), (0.5265, 0.702, 0.0, 0.6898132983950213, -0.3097472122226715, 0.351, -0.29800000000000004, 0.22762933333333332, 0.6639155654456877), (0.543, 0.724, 0.0, 0.7038237200259235, -0.2949768623238631, 0.362, -0.276, 0.18543333333333334, 0.6476782879745575), (0.526, 0.7013333333333334, 0.0, 0.6893840104849279, -0.3101933772205294, 0.3506666666666667, -0.29866666666666664, 0.1557386666666667, 0.629586049653622

[ 49   1   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 0.99  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 0.98  
Training/ Validation / Test MCC: 1.0 / 1.0 / 0.986666070043373  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.511, 0.6813333333333333, 0.0, 0.6763732627399074, -0.3235534199997232, 0.3406666666666667, -0.31866666666666665, 0.34465866666666667, 0.7405451144119135), (0.5025, 0.67, 0.0, 0.6688851913477537, -0.33111108633661995, 0.335, -0.32999999999999996, 0.303504, 0.6982511083764631), (0.51, 0.68, 0.0, 0.6754966887417219, -0.3244428422615251, 0.34, -0.31999999999999995, 0.41412000000000004, 0.7789909883823901), (0.5065, 0.6753333333333333, 0.0, 0.6724195154331232, -0.32755511771960594, 0.33766666666666667, -0.32466666666666666, 0.375896, 0.7489890160354069), (0.5125, 0.6833333333333333, 0.0, 0.6776859504132231, -0.32221908170066815, 0.3416666666666667, -0.31666666666666665, 0.33031333333333335, 

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.697, 0.9293333333333333, 0.0, 0.8214496169711255, -0.13658478329266088, 0.4646666666666667, -0.07066666666666666, 0.008134666666666672, 0.5390228198424123), (0.6975, 0.93, 0.0, 0.8217967599410898, -0.13590311619710724, 0.465, -0.06999999999999995, 0.0074840000000000045, 0.5389657618585564), (0.7195, 0.9593333333333334, 0.0, 0.8368711834835708, -0.10240363797904162, 0.4796666666666667, -0.04066666666666663, 0.006773333333333339, 0.5388546581954994), (0.6935, 0.9246666666666666, 0.0, 0.8190138765869501, -0.14128389012267617, 0.4623333333333333, -0.07533333333333336, 0.0062746666666666714, 0.5387732478390692), (0.688, 0.9173333333333333, 0.0, 0.8151658767772512, -0.1484341391812703, 0.45866666666666667, -

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5135, 0.6846666666666666, 0.0, 0.6785596299966964, -0.32132937142774914, 0.3423333333333333, -0.31533333333333335, 0.0021026666666666685, 0.538235536820226), (0.51, 0.68, 0.0, 0.6754966887417219, -0.3244428422615251, 0.34, -0.31999999999999995, 0.0025533333333333354, 0.5382473970234588)]
[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5135, 0.6846666666666666, 0.0, 0.6785596299966964, -0.32132937142774914, 0.3423333333333333, -0.31533333333333335, 0.0021026666666666

[ 50   0   0 150]
 
Training/ Validation / Test BACC : 1.0 / 1.0 / 1.0  
Training/ Validation / Test Youden-index: 1.0 / 1.0 / 1.0  
Training/ Validation / Test MCC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUC: 1.0 / 1.0 / 1.0  
Training/ Validation / Test AUPR: 1.0 / 1.0 / 1.0
[(0.5135, 0.6846666666666666, 0.0, 0.6785596299966964, -0.32132937142774914, 0.3423333333333333, -0.31533333333333335, 0.0021026666666666685, 0.538235536820226), (0.51, 0.68, 0.0, 0.6754966887417219, -0.3244428422615251, 0.34, -0.31999999999999995, 0.0025533333333333354, 0.5382473970234588), (0.4975, 0.6633333333333333, 0.0, 0.664440734557596, -0.33555558016542664, 0.33166666666666667, -0.33666666666666667, 0.003913333333333335, 0.5383432693963855), (0.522, 0.696, 0.0, 0.6859395532194481, -0.31376041154824214, 0.348, -0.30400000000000005, 0.0020906666666666686, 0.538240191178545), (0.509, 0.6786666666666666, 0.0, 0.6746189529489728, -0.32533216699935646, 0.3393333333333333, -0.32133333333333336, 0.00239866

In [61]:
perf_bc_bn_pbno3_1.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.30485,0.405333,0.0034,0.466561,-0.514971,0.204367,-0.591267,0.204367,0.700456
LR,0.52625,0.665933,0.1072,0.678310,-0.219606,0.386567,-0.226867,0.548323,0.846110
LinSVM,0.50320,0.623600,0.1420,0.652919,-0.219727,0.382800,-0.234400,0.483706,0.811636
MLP,0.52360,0.656933,0.1236,0.674055,-0.210016,0.390267,-0.219467,0.461228,0.773198
NB,0.68735,0.916467,0.0000,0.814705,-0.149223,0.458233,-0.083533,0.457767,0.855995
RBFSVM,0.50960,0.679467,0.0000,0.675140,-0.324798,0.339733,-0.320533,0.428160,0.770456
RF,0.46650,0.597800,0.0726,0.626407,-0.306870,0.335200,-0.329600,0.281262,0.672230


In [62]:
perf_bc_bn_pbno3_2.groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.60175,0.795600,0.0202,0.747794,-0.212542,0.407900,-0.184200,0.407900,0.828125
LR,0.50800,0.677333,0.0000,0.673739,-0.326221,0.338667,-0.322667,0.202504,0.619383
LinSVM,0.50820,0.677600,0.0000,0.673907,-0.326043,0.338800,-0.322400,0.339193,0.726744
MLP,0.52800,0.704000,0.0000,0.690689,-0.308242,0.352000,-0.296000,0.002390,0.538251
NB,0.71810,0.957467,0.0000,0.835920,-0.104711,0.478733,-0.042533,0.476967,0.864770
RBFSVM,0.69920,0.932267,0.0000,0.822946,-0.133083,0.466133,-0.067733,0.006969,0.538903
RF,0.52825,0.704333,0.0000,0.691273,-0.308175,0.352167,-0.295667,0.192851,0.649645


In [63]:
pd.concat([perf_bc_bn_pbno3_1, perf_bc_bn_pbno3_2], axis=0).groupby('Model').mean()

,Accuracy,Sensitivity,Specificity,F1,MCC,BACC,Yoden Index,AUROC,AUPR
Model,,,,,,,,,
DT,0.453300,0.600467,0.0118,0.607177,-0.363757,0.306133,-0.387733,0.306133,0.764291
LR,0.517125,0.671633,0.0536,0.676025,-0.272914,0.362617,-0.274767,0.375414,0.732746
LinSVM,0.505700,0.650600,0.0710,0.663413,-0.272885,0.360800,-0.278400,0.411450,0.769190
MLP,0.525800,0.680467,0.0618,0.682372,-0.259129,0.371133,-0.257733,0.231809,0.655724
NB,0.702725,0.936967,0.0000,0.825313,-0.126967,0.468483,-0.063033,0.467367,0.860383
RBFSVM,0.604400,0.805867,0.0000,0.749043,-0.228941,0.402933,-0.194133,0.217565,0.654679
RF,0.497375,0.651067,0.0363,0.658840,-0.307522,0.343683,-0.312633,0.237057,0.660938
